In [1]:
data_ok = True

In [2]:
DOG_FOLDER = "data/dog_images"
RESIZE = 256
CROP_SIZE = 224
BATCH_SIZE = 20
INPUT_DIM = 3
OUTPUT_DIM = 133
CONV1_OUT_DIM = 8
CONV2_OUT_DIM = 16
CONV3_OUT_DIM = 32
DROP_PROB = 0.5
POOL_KERNEL_SIZE = 2
CONV_KERNEL_SIZE = 3

FLIP_PROB = 0.5
DEGREES = 30

USE_TRANSFER = True

### Upload data to S3

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'dog_classification'

# role = sagemaker.get_execution_role()
role = "arn:aws:iam::115635759328:role/service-role/AmazonSageMaker-ExecutionRole-20210927T084228"

pandas failed to import. Analytics features will be impaired or broken.


In [4]:
if not data_ok:
    !aws s3 cp --recursive {DOG_FOLDER} s3://{bucket}/{prefix}/dog_images

### Training the model

In [5]:
from sagemaker.pytorch import PyTorch

estimator_scratch = PyTorch(entry_point="main.py",
                            source_dir="src",
                            role=role,
                            framework_version="1.4.0",
                            py_version = "py3",
                            train_instance_count=1,
                            train_instance_type="ml.p2.xlarge",
                            hyperparameters={
                                "resize": RESIZE,
                                "crop-size": CROP_SIZE,
                                "batch-size": BATCH_SIZE,
                                "epochs": 20,
                                "input-dim": INPUT_DIM,
                                "conv1-out-dim": CONV1_OUT_DIM,
                                "conv2-out-dim": CONV2_OUT_DIM,
                                "conv3-out-dim": CONV3_OUT_DIM,
                                "conv-kernel-size": CONV_KERNEL_SIZE,
                                "pool-kernel-size": POOL_KERNEL_SIZE,
                                "drop-prob": DROP_PROB,
                                "flip-prob": FLIP_PROB,
                                "degrees": DEGREES,
                                "output-dim": OUTPUT_DIM,
                                "use-transfer": USE_TRANSFER,
                            })

In [6]:
inputs = {
    'training': f"s3://{bucket}/{prefix}/dog_images/train",
    'validation': f"s3://{bucket}/{prefix}/dog_images/valid",
    'testing': f"s3://{bucket}/{prefix}/dog_images/test"
    }

In [7]:
estimator_scratch.fit(inputs)

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-12-03 12:34:37 Starting - Starting the training job...
2021-12-03 12:34:40 Starting - Launching requested ML instances.........
2021-12-03 12:36:44 Starting - Preparing the instances for training............
2021-12-03 12:38:33 Downloading - Downloading input data............
2021-12-03 12:40:57 Training - Downloading the training image...
2021-12-03 12:41:29 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-12-03 12:41:30,124 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-12-03 12:41:30,152 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-12-03 12:41:33,170 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-12-03 12:41:33,536 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setu

In [ ]:
predictor_scratch = estimator_scratch.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

In [ ]:
result = predictor_scratch.predict("data/teste.jpg")

In [ ]:
predictor_scratch.delete_endpoint()